In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy
! pip install fasttext
!pip install git+https://github.com/alexyorke/butter-fingers#egg=butterfingers
import fasttext
import butterfingers
from google.colab import drive
from datetime import datetime
import os
from random import randint
import random
import time
drive.mount('/content/gdrive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.7 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3163014 sha256=1af41d758ef4abd31c0521f2b0ff769d012df0757ad14b932aad013617b56b40
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/alexyorke/butter-fingers to /tmp/pip-install-d8q7xg26/butterfingers_70983d456d2d41988eb423eae521369a
  Running command git clone -q https://github.com/alexyorke/butter-fingers /tmp/pip-install-d8q7xg26/butterfingers_70983d456d2d41988eb423eae521369a
  Created wheel for butterfingers: filename=butterfingers-0.1-py3-none-any.whl size=2880 sha256=dcdfd19be6b35

# 1 Load Data 

In [3]:
def load_data_2_compare(path_A, path_B):
  dataA = pd.read_csv(path_A, header = 0, engine = 'c').fillna(' ')
  dataB = pd.read_csv(path_B, header = 0, engine = 'c').fillna(' ')
  dataA = dataA.astype(str).apply(lambda x:x.name.replace('_',' ') + ': ' + x)
  dataB = dataB.astype(str).apply(lambda x:x.name.replace('_',' ') + ': ' + x)
  return dataA, dataB

# 2 Typo generation

In [6]:
def get_typo_lists(str_df, num=10, prob=0.1):
  typolist = []
  for n in range(0,num):
    df_proc = deepcopy(str_df)
    df_proc = df_proc[df_proc.columns[1:]]
    for col in df_proc.columns:
      df_proc[col]=df_proc[col].apply(butterfingers, args=(prob, 'querty'))
    df_proc[str_df.columns[0]]=str_df[str_df.columns[0]]
    typolist.append(df_proc)
  return typolist

In [7]:
def create_voc_iter(iter_lists, ori_l, ori_r, path_base):
  for iter in iter_lists:
    if iter == 0:
      df_full = pd.concat([ori_l, ori_r])
    else:
      typolist_l1 = get_typo_lists(ori_l, iter, 0.05)
      typolist_r1 = get_typo_lists(ori_r, iter, 0.05)
      typolist_l1.append(ori_l)
      typolist_r1.append(ori_r)
      df_typo_l1 = pd.concat(typolist_l1)
      df_typo_l1.reset_index(drop=True, inplace=True)
      df_typo_r1 = pd.concat(typolist_r1)
      df_typo_r1.reset_index(drop=True, inplace=True)
      df_full = pd.concat([df_typo_l1, df_typo_r1])
    filename = 'datastring_' + str(iter) + '.txt'
    path = os.path.join(path_base, filename)
    df_full.to_csv(path, sep = ' ', header = False, index = False)
    print('store: ' + filename + ' complete')

# 3 Train a fasttext embedding 

In [8]:
def train_embeddings(path_base, iter_lists):
  for iter in iter_lists:
    start = time.time()
    filename = 'datastring_' + str(iter) + '.txt'
    path = os.path.join(path_base, filename)
    emb_temp=fasttext.train_unsupervised(path,
                                         'skipgram',loss='hs',minn=2, maxn=4, dim=100)
    model_name='walmart_amazon_'+str(iter)+'.en.bin'
    model_path = os.path.join(path_base, model_name)
    emb_temp.save_model(model_path)
    end = time.time()
    duration = end-start
    print('embedding for model ' + str(iter) +' is_saved')
    print('time spent: '+ str(duration))

# 5 compare different embeddings

In [9]:
from zipfile import ZipFile

In [ ]:
emb_10 = ZipFile('/content/drive/My Drive/Colab Notebooks/CSC 2508 Project/Data/itunes_amazon/Embeddings/itunes_amazon.en.zip')
emb_10.extractall(path='/content/drive/My Drive/Colab Notebooks/CSC 2508 Project/Data/itunes_amazon/Embeddings')

In [ ]:
for i in [0,15,20,25]:
  path_temp = '/content/drive/My Drive/Colab Notebooks/CSC 2508 Project/Data/itunes_amazon/Embeddings/itunes_amazon'+str(i)+'.en.zip'
  emb_temp = ZipFile(path_temp)
  emb_temp.extractall(path='/content/drive/My Drive/Colab Notebooks/CSC 2508 Project/Data/itunes_amazon/Embeddings')

## Iteration 10

In [ ]:
model_10 = fasttext.load_model(path='/content/drive/My Drive/Colab Notebooks/CSC 2508 Project/Data/itunes_amazon/Embeddings/itunes_amazon.en.bin')

In [ ]:
model_10.get_nearest_neighbors('midnight')

[(0.8838357329368591, 'Lidnight'),
 (0.8766768574714661, 'Oidnight'),
 (0.8672534227371216, 'Nidnight'),
 (0.8668590188026428, 'Iidnight'),
 (0.8428350687026978, 'Jidnight'),
 (0.834240198135376, 'Kidnight'),
 (0.8256857991218567, 'Midnighc'),
 (0.8237775564193726, 'Midnighh'),
 (0.8217277526855469, 'Midnight'),
 (0.8212246894836426, 'Midnighy')]

## Iteration 15

In [ ]:
model_15 = fasttext.load_model(path='/content/drive/My Drive/Colab Notebooks/CSC 2508 Project/Data/itunes_amazon/Embeddings/itunes_amazon15.en.bin')

In [ ]:
model_15.get_nearest_neighbors('musik')

[(0.7308444380760193, 'Jusik'),
 (0.7113716006278992, 'Kusik'),
 (0.6793750524520874, 'Musik'),
 (0.6727420687675476, 'musik"'),
 (0.6546809077262878, 'Dausik'),
 (0.6430073380470276, 'Einmusik'),
 (0.6187251210212708, 'Mjsik'),
 (0.5834507942199707, 'music"'),
 (0.5510935187339783, 'Frankmusik'),
 (0.5432460904121399, 'must')]